<a href="https://colab.research.google.com/github/gzsomm/Naive-Bayes/blob/main/Proyecto_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Carga de base de datos**

In [32]:
# Cargo paquetes a utilizar
import pandas as pd
import numpy as nop
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# Cargo base de datos y transformo a DataFrame
bbdd = pd.read_csv ('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv')
df = pd.DataFrame(bbdd)
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [34]:
df.shape

(891, 3)

In [35]:
# Elimino variable 'package_name'
df.drop('package_name', axis=1, inplace=True)
df.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [36]:
# Transformo variable review
df["review"] = df["review"].str.strip().str.lower()

**Entrenamiento del modelo**

In [37]:
# Divido data frame para entrenamiento

from sklearn.model_selection import train_test_split

X = df["review"]
y = df["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

,review
331,just did the latest update on viber and yet ag...
733,keeps crashing it only works well in extreme d...
382,the fail boat has arrived the 6.0 version is t...
704,"superfast, just as i remember it ! opera mini ..."
813,installed and immediately deleted this crap i ...


In [38]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")

X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()


In [39]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
# Construyo el modelo Naive
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [41]:
from sklearn.metrics import accuracy_score

y_pred_mnb = model.predict(X_test)
acc_mnb = accuracy_score(y_test, y_pred_mnb)
print(f'Precisión del modelo: {acc_mnb:.4f}')

Precisión del modelo: 0.8156


In [42]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB

for model_aux in [GaussianNB(), BernoulliNB()]:
    model_aux.fit(X_train, y_train)
    y_pred_aux = model_aux.predict(X_test)
    print(f"{model_aux} with accuracy: {accuracy_score(y_test, y_pred_aux)}")

GaussianNB() with accuracy: 0.8044692737430168
BernoulliNB() with accuracy: 0.770949720670391


El mejor modelo es el de Mutimodal según se confirma en los datos de precisión lograda. Esto confirma los supuestos teóricos vistos en el programa.

In [43]:
# Mejora del modelo
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200),
    "fit_prior": [True, False]
}

# We initialize the random search
random_search = RandomizedSearchCV(model, hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=50,
                   param_distributions={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.1646231...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                                        'fit_prior': [True, False]},
                   random_state=42, scoring='accuracy')

In [44]:
random_search.fit(X_train, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")

Best hyperparameters: {'fit_prior': False, 'alpha': np.float64(1.917638190954774)}


In [45]:
model = MultinomialNB(alpha = 1.917638190954774, fit_prior = False)
model.fit(X_train, y_train)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_mnbh = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {acc_mnbh:.4f}')

Precisión del modelo: 0.8212


Se constata una mejora en la precisión del modelo modificando sus hiperpárametros

In [46]:
# Guardo el modelo
from pickle import dump

dump(model, open("/models/naive_bayes_alpha_1-9176382_fit_prior_False_42.sav", "wb"))

FileNotFoundError: [Errno 2] No such file or directory: '/models/naive_bayes_alpha_1-9176382_fit_prior_False_42.sav'